In [1]:
pip install transformers datasets sentencepiece

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 325 kB 53.1 MB/s 
     |████████████████████████████████| 1.2 MB 54.4 MB/s 
     |████████████████████████████████| 6.5 MB 48.8 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 76.7 MB/s 
     |████████████████████████████████| 895 kB 70.4 MB/s 
     |████████████████████████████████| 212 kB 76.2 MB/s 
     |████████████████████████████████| 134 kB 76.3 MB/s 
     |████████████████████████████████| 1.1 MB 65.5 MB/s 
     |████████████████████████████████| 127 kB 66.0 MB/s 
     |████████████████████████████████| 144 kB 49.7 MB/s 
     |████████████████████████████████| 271 kB 56.9 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
model_name ='asafaya/bert-mini-arabic'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/326k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
quranpath = '/content/drive/MyDrive/Quran_QA/trials/Embedding_corpus/corpus.txt'
with open(quranpath , 'r' , encoding= 'utf-8') as f:
    lines = f.readlines()

In [5]:
for i in range(len(lines)):
  lines[i] = lines[i][:-1]

In [6]:
import numpy as np

maxlength =  max([len(x.split()) for x in lines])
maxlengthid =  np.argmax([len(x.split()) for x in lines])
maxlength

198

In [7]:
lines[maxlengthid]

'يا أيها الذين أمنوا إذا تداينتم ب دين إلى أجل مسمى ف أكتبوا ه و ل يكتب بين كم كاتب ب ال عدل و لا يأب كاتب أن يكتب كما علم ه الله ف ل يكتب و ل يملل الذي على ه ال حق و ل يتق الله رب ه و لا يبخس من ه شيئا ف إن كان الذي على ه ال حق سفيها أو ضعيفا أو لا يستطيع أن يمل هو ف ل يملل ولي ه ب ال عدل و استشهدوا شهيدين من رجال كم ف إن لم يكونا رجلين ف رجل و امرأتان من من ترضون من ال شهداء أن تضل أحدا هما ف تذكر أحدا هما ال أخرى و لا يأب ال شهداء إذا ما دعوا و لا تساموا أن تكتبوا ه صغيرا أو كبيرا إلى أجل ه ذلكم أقسط عند الله و أقوم ل ال شهادة و أدنى أن لا ترتابوا إلا أن تكون تجارة حاضرة تديرون ها بين كم ف ليس على كم جناح أن لا تكتبوا ها و أشهدوا إذا تبايعتم و لا يضار كاتب و لا شهيد و إن تفعلوا ف إن ه ف سوق ب كم و اتقوا الله و يعلم كم الله و الله ب كل شيء عليم'

In [8]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': [quranpath], 'test':quranpath})

Using custom data configuration default-af0be85310bbb431


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-af0be85310bbb431/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6236
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6236
    })
})

In [10]:

def tokenize_pad_and_truncate(texts):
    return tokenizer(texts["text"], padding="max_length")

tokenized_datasets = dataset.map(tokenize_pad_and_truncate, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


  0%|          | 0/7 [00:00<?, ?ba/s]

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.2)

In [12]:
from transformers import  TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy='no',
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.001,
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'] ,
    eval_dataset = tokenized_datasets['test'],
    data_collator=data_collator,
)

In [15]:
len(tokenized_datasets['train'])

6236

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6236
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6236


Epoch,Training Loss,Validation Loss
1,3.003700,2.859742
2,2.929600,2.893972
3,3.025200,2.846948


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6236
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6236
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6236
  Batch size = 4


In [17]:
model.save_pretrained("'/content/drive/MyDrive/Quran_QA/quranBERT/mini1")

Configuration saved in '/content/drive/MyDrive/Quran_QA/quranBERT/mini1/config.json
Model weights saved in '/content/drive/MyDrive/Quran_QA/quranBERT/mini1/pytorch_model.bin


In [33]:
sample = tokenizer([""] , return_tensors='pt')
result =model(**sample)
result

MaskedLMOutput([('logits',
                 tensor([[[ -8.5806,  -3.1739,  -4.0091,  ...,  -7.8281,  -9.6646,  -8.3176],
                          [ -9.4118,  -0.0268,  -2.2554,  ...,  -9.0816, -18.8325, -12.5036],
                          [-12.1986,  -1.3682,  -6.6483,  ...,  -9.2725, -10.6983,  -9.8014],
                          ...,
                          [ -9.5398,   2.1681,  -6.6413,  ...,  -7.3313,  -5.9929,  -7.8964],
                          [ -8.9356,  -3.1840,  -5.2747,  ...,  -7.6708,  -7.1048,  -8.4862],
                          [ -8.0158,  -3.3436,  -4.1887,  ...,  -7.4730,  -8.7690,  -8.0123]]],
                        grad_fn=<AddBackward0>))])

In [34]:
import numpy as np

ids =np.argmax(result[0].detach().numpy(), axis= 2)
print(ids)
tokenizer.decode(ids[0])

[[ 244 2723 3519 1716 2164 1724 1809]]


'ف انا لله اليكين ما'